<center>
<img src="../../img/ods_stickers.jpg" />
    
## [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 
Author: [Yury Kashnitskiy](https://yorko.github.io) (@yorko). This material is subject to the terms and conditions of the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license. Free use is permitted for any non-commercial purpose. This material is a translated version of the Capstone project (by the same author) from specialization "Machine learning and data analysis" by Yandex and MIPT. No solutions shared.

# <center>Week 5. "Catch Me If You Can" Kaggle Inclass Competition

This week we will refresh our knowledge about the concept of stochastic gradient descend and try Scikit-learn `SGDClassifier` which works much faster on large amounts of data than algorithms we used in week 4. Also, we are going to explore the data in Kaggle inclass [competition](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) and to make the a couple of submissions.

**Disclaimer:** this part is mostly duplicated by assignment 4 in the course.

**Your task**
1. Fill in code in provided notebook
2. Choose answers in the [form](https://docs.google.com/forms/d/1UUJOcnN5ZSjJ4XAfta6_S468JNVn-Ph7uCh6wTR-YEY)

In [1]:
# Disable Anaconda warnings
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score

**Read [competition](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) data into train_df and test_df DataFrames (train and test datasets).**

In [2]:
# Change the path to data
PATH_TO_DATA = '../../data/'

In [3]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id')

In [4]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


**Concatenate train and test datesets - it is necessary to switch the data to sparse format.**

In [5]:
train_test_df = pd.concat([train_df, test_df])

There are following features in the training dataset:
    - site1 – index of the first website visited in a session
    - time1 – timestamp of the first website visiting in a session
    - ...
    - site10 – index of the 10-th website visited in a session
    - time10 – timestamp of the 10-th website visiting in a session
    - user_id – user id

User sessions are chosen in such a way that they are no longer than half an hour and/or contain more than ten websites i.e. a session is considered ended if either a user has visited ten websites or a session has lasted for more than thirty minutes.

**Let's look at the features statistics.**

**There are some gaps where sessions are short (less than 10 websites). Let's say, if a person visited *vk.com* 2015-01-01 20:01, then this person visited *yandex.ru* at 20:29, then *google.com* at 20:33, then his/her first session consits of just 2 websites (site1 - website ID of *vk.com*, time1 – 2015-01-01 20:01:00, site2 – website ID of *yandex.ru*, time2 – 2015-01-01 20:29:00, other features – NaN), and starting with *google.com* there will be a new session, because there are more than 30 minutes passed since *vk.com* visit.**

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 1 to 253561
Data columns (total 21 columns):
site1     253561 non-null int64
time1     253561 non-null object
site2     250098 non-null float64
time2     250098 non-null object
site3     246919 non-null float64
time3     246919 non-null object
site4     244321 non-null float64
time4     244321 non-null object
site5     241829 non-null float64
time5     241829 non-null object
site6     239495 non-null float64
time6     239495 non-null object
site7     237297 non-null float64
time7     237297 non-null object
site8     235224 non-null float64
time8     235224 non-null object
site9     233084 non-null float64
time9     233084 non-null object
site10    231052 non-null float64
time10    231052 non-null object
target    253561 non-null int64
dtypes: float64(9), int64(2), object(10)
memory usage: 42.6+ MB


In [7]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44


In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82797 entries, 1 to 82797
Data columns (total 20 columns):
site1     82797 non-null int64
time1     82797 non-null object
site2     81308 non-null float64
time2     81308 non-null object
site3     80075 non-null float64
time3     80075 non-null object
site4     79182 non-null float64
time4     79182 non-null object
site5     78341 non-null float64
time5     78341 non-null object
site6     77566 non-null float64
time6     77566 non-null object
site7     76840 non-null float64
time7     76840 non-null object
site8     76151 non-null float64
time8     76151 non-null object
site9     75484 non-null float64
time9     75484 non-null object
site10    74806 non-null float64
time10    74806 non-null object
dtypes: float64(9), int64(1), object(10)
memory usage: 13.3+ MB


**There are 2297 sessions of user Alice and 251264 sessions of other users, who are not Alice. There is a huge class disbalance and it doesn't make good sense to look at accuracy in this case.**

In [9]:
train_df['target'].value_counts()

0    251264
1      2297
Name: target, dtype: int64

**For a prediction we are going to use just indices of visited websites. Indices have been enumerated starting from 1, so replace missed values with zeros.**

In [10]:
train_test_df_sites = train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')

In [11]:
train_test_df_sites.head(10)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,718,0,0,0,0,0,0,0,0,0
2,890,941,3847,941,942,3846,3847,3846,1516,1518
3,14769,39,14768,14769,37,39,14768,14768,14768,14768
4,782,782,782,782,782,782,782,782,782,782
5,22,177,175,178,177,178,175,177,177,178
6,570,21,570,21,21,0,0,0,0,0
7,803,23,5956,17513,37,21,803,17514,17514,17514
8,22,21,29,5041,14422,23,21,5041,14421,14421
9,668,940,942,941,941,942,940,23,21,22


**Create sparse matrices *X_train_sparse* and *X_test_sparse* as we did before. Use concatenated matrix *train_test_df_sites*, then split it back into train and test parts.**

Notice that there are zeros in the sessions which have less than 10 websites, so the first feature (which shows the number of zeros in a session) differs from other features (which shows the number of websites with index $i$ in a session). Therefore we should delete the first column of a sparse matrix.

**Detach labels from training data into a separate vector *y*.**

In [ ]:
train_test_sparse = ''' YOUR CODE IS HERE '''
X_train_sparse = ''' YOUR CODE IS HERE '''
X_test_sparse = ''' YOUR CODE IS HERE '''
y = ''' YOUR CODE IS HERE '''

**<font color='red'>Question 1. </font> Print the dimensions of the matrices *X_train_sparse* and *X_test_sparse* - 4 numbers with spaces between them: rows number and columns number of *X_train_sparse*, then rows number and columns number of *X_test_sparse*.**

In [ ]:
''' YOUR CODE IS HERE '''

**Save *X_train_sparse*, *X_test_sparse* and *y* objects into pickle files (last one – into the file *kaggle_data/train_target.pkl*).**

In [ ]:
with open(os.path.join(PATH_TO_DATA, 'X_train_sparse.pkl'), 'wb') as X_train_sparse_pkl:
    pickle.dump(X_train_sparse, X_train_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'X_test_sparse.pkl'), 'wb') as X_test_sparse_pkl:
    pickle.dump(X_test_sparse, X_test_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'train_target.pkl'), 'wb') as train_target_pkl:
    pickle.dump(y, train_target_pkl, protocol=2)

**Split train dataset into 2 parts in proportion 7:3 (70%,30%) with no shuffle. Original data is sorted by time, test dataset separated from test dataset timewise, follow these rules here as well.**

In [ ]:
train_share = int(.7 * X_train_sparse.shape[0])
X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]

**Create object `sklearn.linear_model.SGDClassifier` with logloss and *random_state*=17. Other parameters leave default, except maybe *n_jobs*=-1 - it almost never hurts. Train model on `(X_train, y_train)`.**

In [ ]:
sgd_logit = ''' YOUR CODE IS HERE '''
sgd_logit.fit ''' YOUR CODE IS HERE '''

**Predict probabilities that sessions belong to Alice for the holdout dataset *(X_valid, y_valid)*.**

In [ ]:
logit_valid_pred_proba = sgd_logit ''' YOUR CODE IS HERE '''

**<font color='red'>Question 2. </font> Evaluate ROC AUC for logistic regression trained with stochastic gradient descend on the holdout dataset. Round your answer up to the third digit after decimal point.**

In [ ]:
''' YOUR CODE IS HERE '''

**Predict probabilities of class 1 for test dataset with *sgd_logit* trained on the whole train dataset (not only on 70% of train data).**

In [ ]:
%%time
sgd_logit ''' YOUR CODE IS HERE '''
logit_test_pred_proba = ''' YOUR CODE IS HERE '''

**Write your predictions to a file and make a submission at Kaggle. You've a done a very simple submission, now you can improve your model with additional features.**

In [ ]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
write_to_submission_file ''' YOUR CODE IS HERE '''

**In week 6, we will walk through a large Vowpal Wabbit tutorial and try it in practice on websites visits data.**